<a href="https://colab.research.google.com/github/lej8924/samsung-digital-it/blob/main/HyeongSoo/RAG_MultiQuery_ParentInfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall typer -y

Found existing installation: typer 0.9.4
Uninstalling typer-0.9.4:
  Successfully uninstalled typer-0.9.4


In [2]:
!pip install typer<0.10.0
!pip install spacy weasel --upgrade
!pip install sentence-transformers chromadb openai --upgrade

/bin/bash: line 1: 0.10.0: No such file or directory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: weasel
    Found existing installation: weasel 0.3.4
    Uninstalling weasel-0.3.4:
      Successfully uninstalled weasel-0.3.4
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.4
    Uninstalling spacy-3.7.4:
      Successfully uninstalled spacy-3.7.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.6 MB/s eta 0:00:00


In [3]:
# langchain 및 관련 모듈 설치
!pip install langchain
!pip install langchain_community
!pip install langchain_core
!pip install langchain_experimental

# 추가적으로 필요한 모듈 설치
!pip install faiss-cpu  # FAISS 설치
!pip install PyPDF2  # PDF 로더를 위한 설치
!pip install chainlit  # Chainlit 설치
!pip install bm25  # BM25 Retriever 설치

# 추가적인 필요한 모듈 설치
!pip install ollama

# 필요한 텍스트 분할기 및 임베딩 관련 설치
!pip install sentence-transformers
!pip install chromadb
!pip install transformers  # 필요한 경우
!pip install torch  # PyTorch 설치
!pip install tqdm  # 프로그레스 바 설치

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

## Multi-Query

In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.retrievers  import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain.chains import RetrievalQA
import time
import chainlit as cl
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chat_models import ChatOpenAI
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline,AutoModelForCausalLM
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import ParentDocumentRetriever

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!pip install PyPDF2
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 7.9 MB/s eta 0:00:00


In [11]:
pdf_filepath = '/content/drive/MyDrive/삼성증권/Data/test_samsung.pdf' # 파일 경로 알아서 해야함
loader = PyPDFLoader(pdf_filepath)
pages = loader.load()
print(pages)
print("=====loader=====")

[Document(page_content=" \n \n퀄컴  (QCOM  US) \nFY 4Q23 review  – 기대 반, 걱정 반 \n \n• 실적과  가이던스  모두 컨센서스  상회. 마침내  업황 개선과  매출 성장 재개를  전망 \n• 한편 모바일  점유율  감소, 신사업 (Arm/AI PC ) 경쟁 심화 가능성은 우려 요인 \n• 투자 심리는 마침내  돌아서겠지만  아직 섹터 내 outperform 에 대한 믿음은  부족 \nWHAT’S THE STORY ? \nFY 4Q23 review : 4분기 컨센서스를  상회하는  데 성공. Non-GAAP 기준, 매출액은  전년 동기 \n대비 24% 감소, 전분기  대비 3% 증가. QCT(반도체 )와 QTL(라이선스 ) 매출액  모두 전분기  \n대비 3% 성장. 세부적으로 , Automotive 가 가장 가파른  성장률 (23% q -q)을 기록했고 , 가장 \n비중이  큰 Handsets도 4% 성장. IoT(-7%)의 재고 조정은  지속되는  모습. 지난 분기를  고점\n으로 재고자산  회전 일수도  마침내  감소 (157일 → 149일). \n차기 분기 가이던스 : 차기 분기 가이던스도 기대 상회. 매출 전망 91~99억 달러는 전분기  \n대비 5~14% 성장하는  수준. 참고로  컨센서스 (FactSet)는 91.9억 달러. 중간값  기준으로 , 마\n침내 전년 대비 성장하는  것을 전망했다는  점도 고무적 . Non-GAAP EPS 가이던스  \n2.25~2.45 달러도  컨센서스  2.22달러 상회.  \n드디어  모바일  수요 안정화  조짐이  보인다는  입장. 일례로  중화권  업체향  매출액이  전분기  \n대비 35% 증가할  것으로  전망했음 . 올해(CY 2023)  핸드셋  시장 전망도 변경 (높은 한 자릿\n수 감소 → 한 자릿수  중반~후반 사이 감소). 한편 Automotive 와 IoT는 동 기간 모두 감소 \n예정. \nAI가 만들 upside : 사이클  회복만큼이나

In [13]:
# 자식 문서 텍스트 분할 설정 (chunk_size=150)
child_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=30,
    length_function=len,
)
child_chunks = child_text_splitter.split_documents(pages)

# 부모 문서 텍스트 분할 설정 (chunk_size=600)
parent_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=50,
    length_function=len,
)
parent_chunks = parent_text_splitter.split_documents(pages)

# 임베딩 모델 생성
embeddings_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sroberta-multitask',
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True},
)

In [14]:
# 임베딩 모델 생성
embeddings_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sroberta-multitask',
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True},
)

# FAISS 벡터 스토어 생성
child_vectorstore = FAISS.from_documents(child_chunks, embeddings_model)

# 자식 문서를 위한 FAISS 벡터 스토어 생성 및 저장
child_vectorstore = FAISS.from_documents(child_chunks, embeddings_model)
child_vectorstore.save_local("/content/drive/MyDrive/삼성증권/FAISS/child_vectorstore")

In [15]:
# 부모 문서를 위한 FAISS 벡터 스토어 생성
parent_vectorstore = FAISS.from_documents(parent_chunks, embeddings_model)

In [17]:
!pip install rank_bm25

In [18]:
# BM25 Retriever 생성
bm25_retriever = BM25Retriever.from_texts([chunk.page_content for chunk in child_chunks])

In [19]:
# 허깅페이스 Llama 모델 불러오기
model_name = "/content/drive/MyDrive/삼성증권/Llama"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
text2text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [20]:
# 멀티쿼리 생성 함수 정의
def generate_multi_query(query, text2text_generator):
    input_text = f"질문: {query}"
    result = text2text_generator(input_text, max_length=256, num_return_sequences=3)
    queries = [output['generated_text'] for output in result]
    return queries

In [21]:
# 질문 입력 및 멀티쿼리 실행
question = input("질문을 입력하세요: ")
queries = generate_multi_query(question, text2text_generator)

all_results = []

# 각 멀티쿼리에 대해 검색 수행 및 결과 수집
for query in queries:
    # 자식 문서 검색
    child_results = child_vectorstore.similarity_search(query, k=3)

    # 부모 문서 검색
    for child_doc in child_results:
        parent_results = parent_vectorstore.similarity_search(child_doc.page_content, k=1)
        for result in parent_results:
            similarity_score = result.metadata.get('score', 0)
            all_results.append({
                'content': result.page_content,
                'score': similarity_score
            })


질문을 입력하세요: 다음의 자료는 퀄컴 자료에 대한 것이 맞니??


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:144783 for open-end generation.


In [22]:
# FAISS와 BM25 앙상블 기법을 이용하여 점수 계산 및 상위 2개 문서 선택
ensemble_retriever = EnsembleRetriever(
    retrievers=[child_vectorstore.as_retriever(), bm25_retriever],
    weights=[0.6, 0.4],  # FAISS와 BM25에 각각 가중치를 부여 (필요에 따라 조정 가능)
)

In [23]:
# 전체 결과에서 최종 상위 2개 문서 선택
sorted_results = sorted(all_results, key=lambda x: x['score'], reverse=True)
top_results = sorted_results[:2]

# 최종 상위 2개 문서 출력
for result in top_results:
    print(result['content'])

기타 nm nm nm nm nm nm nm nm nm nm 
총계 40.3  38.1  35.6  36.4  33.5  30.8  28.8  30.2  37.6  30.9  
참고: 회계 연도 (9월 결산) 기준; * Qualcomm CDMA Technologies; ** Qualcomm T echnology Licensing  
자료: Qualcomm
05001,0001,5002,0002,5003,0003,5004,0004,5005,000
1Q13 3Q14 1Q16 3Q17 1Q19 3Q20 1Q22 3Q23
영업이익 (좌측) 영업이익률 (우측)(백만달러 ) (%)
01,0002,0003,0004,0005,0006,0007,0008,0009,00010,000
1Q21 3Q21 1Q22 3Q22 1Q23 3Q23 1Q24E*
핸드셋반도체 라이선스(백만달러 )
05001,0001,5002,0002,5003,000
1Q21 3Q21 1Q22 3Q22 1Q23 3Q23 1Q24E*
차량반도체 IoT 반도체(백만달러 )
